### **TODO:** Also extract additional links from the `links` keys

In [101]:
import pandas as pd
import os
import ijson
import json
import gzip
import itertools
import sys
sys.path.append("../../src/data")
from content_api_extract import extract_link_types
from extract_text_utils import get_text
from datetime import datetime
from collections import Counter
from pprint import pprint
import numpy as np

### Directories

In [ ]:
DATA_DIR = os.getenv("DATA_DIR")
content_api = os.path.join(DATA_DIR, "content_api")
filename = os.path.join(content_api, "11-02-19", "content.json.gz")
filename_labelled = os.path.join(content_api, "11-02-19", "labelled.csv.gz")

In [90]:
labelled = pd.read_csv(filename_labelled, compression="gzip")

/Users/felisialoukou/.pyenv/versions/3.6.0/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [91]:
labelled.columns

Index(['base_path', 'content_id', 'description', 'document_type',
       'first_published_at', 'locale', 'primary_publishing_organisation',
       'publishing_app', 'title', 'body', 'combined_text', 'taxon_id',
       'taxon_base_path', 'taxon_name', 'level1taxon', 'level2taxon',
       'level3taxon', 'level4taxon', 'level5taxon'],
      dtype='object')

### Iteratively extract links from a content store dump

In [141]:
links_keys = ['organisations', 
 'policy_areas', 
 'document_collections', 
 'people', 
 'roles', 
 'speaker', 
 'finder', 
 'ministers', 
 'children', 
 'lead_organisations', 
 'related_guides', 
 'topical_events', 
 'parent', 
 'topics', 
 'mainstream_browse_pages', 
 'related_statistical_data_sets', 
 'sections', 
 'field_of_operation', 
 'related_mainstream', 
 'related_mainstream_content', 
 'part_of_step_navs', 
 'related_to_step_navs', 
 'ordered_related_items_overrides', 
 'pages_part_of_step_nav', 
 'pages_related_to_step_nav', 
 'related_policies']

In [149]:
def get_other_links(content_item):
    link_dict = {}
    for key in links_keys:
        if key in content_item['links'].keys():
            link_dict[key] = [item['base_path'] for item in content_item['links'][key] if "base_path" in item.keys()]
        else:
            link_dict[key] = []
    return link_dict

In [150]:
filename

'/Users/felisialoukou/Documents/govuk-network-embedding/data/content_api/11-02-19/content.json.gz'

In [ ]:
rowlist = []
with gzip.open(filename, "rt") as reader:
    page_links = {}
    related_page_links = {}
    collection_links = {}
    print("Starting...",datetime.now().strftime("%H:%M:%S"))
    content_generator = ijson.items(reader, prefix='item')
    for i, content_item in enumerate(content_generator):
        extract_link_types(collection_links, content_item, related_page_links, page_links)
        row = {"base_path": content_item["base_path"],
               "content_id": content_item["content_id"],
               "title": content_item["title"],
               "keys": content_item["links"].keys(),
               "description": content_item["description"],
               "document_type": content_item["document_type"],
               "first_published_at": content_item["first_published_at"], 
               "locale": content_item["locale"], 
               "primary_publishing_organisation": content_item["primary_publishing_organisation"] \
                                       if "primary_publishing_organisation" in content_item.keys() else np.NaN,
               "publishing_app": content_item["publishing_app"],
               "body": get_text(content_item["details"]),
               "embedded_links": page_links[content_item["base_path"]],
               "related_links":  related_page_links[content_item["base_path"]],
               "collection_links": collection_links[content_item["base_path"]]}
        row.update(get_other_links(content_item))
        rowlist.append(row)
        if i%1000==0:
            print(datetime.now().strftime("%H:%M:%S"),i)
            
print("Ending...",datetime.now().strftime("%H:%M:%S"))
df_extract = pd.DataFrame(rowlist)

Starting... 11:24:10
11:24:10 0
11:24:22 1000


In [ ]:
df_extract.head()

### Look at all possible values in the the links schema

In [56]:
df['keys'] = df['keys'].map(list)

In [65]:
all_keys = Counter([k for key in df['keys'].values for k in key ])

In [71]:
pprint(list(all_keys.items()))

[('available_translations', 237225),
 ('organisations', 226664),
 ('policy_areas', 150138),
 ('primary_publishing_organisation', 223339),
 ('taxons', 208686),
 ('world_locations', 22751),
 ('document_collections', 52778),
 ('original_primary_publishing_organisation', 44408),
 ('people', 34036),
 ('roles', 32781),
 ('speaker', 7769),
 ('finder', 40105),
 ('suggested_ordered_related_items', 30156),
 ('worldwide_organisations', 10324),
 ('ministers', 26262),
 ('children', 16470),
 ('lead_organisations', 4413),
 ('related_guides', 1086),
 ('topical_events', 2830),
 ('parent', 13281),
 ('topics', 12757),
 ('mainstream_browse_pages', 1839),
 ('ordered_related_items', 1970),
 ('related_statistical_data_sets', 566),
 ('documents', 3945),
 ('sections', 78),
 ('meets_user_needs', 751),
 ('field_of_operation', 487),
 ('related_mainstream', 239),
 ('related_mainstream_content', 312),
 ('part_of_step_navs', 170),
 ('related_to_step_navs', 69),
 ('ordered_related_items_overrides', 25),
 ('supporting

In [73]:
df.shape, df.content_id.nunique()

((237234, 9), 237234)

### Save out preprocessed version of `content.json.gz`

In [148]:
out_filename = os.path.join(content_api, "11-02-19", "content_json.csv.gz")
df_extract.drop("keys", axis=1, inplace=True)
df_extract.to_csv(out_filename, compression="gzip", index=False)

NameError: name 'df_extract' is not defined

In [81]:
df2 = pd.DataFrame(df.content_id, columns=["content_id"])

In [84]:
df2.to_csv(os.path.join(content_api, "11-02-19", "content_content_ids.csv.gz"), 
                     index=False, compression="gzip")

In [85]:
df2 = pd.read_csv(os.path.join(content_api, "11-02-19", "content_content_ids.csv.gz"), compression="gzip")

In [87]:
df2

,content_id
0,04a0cc0d-0b9f-45ad-bf57-7c54cbab9df9
1,a61985b0-d6eb-4cf1-8140-642b9557ce00
2,d569ef4b-d632-49a0-9795-6a7ea934b799
3,5e0fee54-7631-11e4-a3cb-005056011aef
4,581cabaf-2ed1-4411-8d1b-35b1bcb559b5
5,781ebba8-3c5b-40d2-8f39-b413d2665272
6,01401c28-d55a-4308-896c-c71285199017
7,dae6d3c3-f995-42e9-bd81-a5dffd7e812d
8,e618840d-40a5-4b80-9008-6b1f8c3f8741
9,5f12a87c-7631-11e4-a3cb-005056011aef
